# 한 키워드 내의 포스트 간의 TF-IDF

## 1. 크롤링 된 전체 text를 각각 포스트 별로 분할

In [77]:
import pandas as pd
import numpy as np

In [78]:
text1 = open( '킹크랩 섹션.txt',encoding='utf-8').read()

#### 이후의 작업을 하는데 있어서 텍스트 자체가 비어있는 경우 때문에 계속적으로 오류가 났기 때문에 그것을 제거해줌.

In [141]:
post_list=text1.split("-----------------------------------")

for i in range(len(post_list)) :
    if(len(post_list[i]) == 0):
        print(i)

20
56
92
108


## 2. konlp의 트위터 형태소 분석기를 이용해서 1차적으로 stopword 제거

In [80]:
from konlpy.tag import *
import time
okt = Okt()

In [81]:
# 트위터 nouns 
okt_result11 = okt.nouns(text1)
text11=' '.join(okt_result11)


In [82]:
removed_post_1st=[]
for post in post_list :
    post_result1=okt.nouns(post)
    post_result2=' '.join(post_result1)
    removed_post_1st.append(post_result2)

## 3. stopword list를 이용해서 2차적으로 stopword 제거

In [83]:
stopwords=pd.read_csv("한글불용어.csv",encoding='cp949')
stopword=stopwords["불용어"].tolist()

def remove_stopwords(text):
    tokens = text.split(' ')
    stops = stopword
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words)

In [84]:
removed_post_2nd=[]
for post in removed_post_1st :
    removed_post=remove_stopwords(post)
    removed_post_2nd.append(removed_post)

In [85]:
print(removed_post_2nd)

['거성 수산 판매 킹크랩 섹션 러시아 캄차카 해역 포획 하자 마자 배 킹크랩 손질 자숙 배 급속 선동 제품 율과 선도 발생 여유 제품 국내 반입 제품 킹크랩 부위 부위 모두 제거 무게 이기 킹크랩 선어 가격 면 더욱 킹크랩 섹션 킹크랩 최고 부위 다릿살 옆구리 물 가정 용 찜솥 킹크랩 섹션 판매 가격 키 당 키 로부터 무료 택배 미만 추가 박스 제품 주문 전화 문자 카톡 거성 수산 킹크랩 판매 전국 택배 고속버스 탁송', '어제 홈쇼핑 크랩 방송 이번 첫 킹크랩 방송 진행 전시 킹크랩 먹음 직 킹크랩 실제 고객 주문 킹크랩 섹션 물 킹크랩 모습 방송 준비 위해 정리 대표 모습 크랩 전문가 모습 모든 준비 마치 고객 오늘 방송 최 설례 쇼핑 호스트 대표 니 오늘 킹크랩 방송 대표 모습 진행 킹크랩 홈쇼핑 진행 킹크랩 역대 가장 사이즈 홈쇼핑 진행 킹크랩 제품 준비 호스트 깜짝 사이즈 준비 보시 얼마만큼 예전 준비 상품 제품 를 위해 준비 킹크랩 다리 살이 쇼 호스트 한번 사이즈 대표 집게발 꽉 찬 살이 분 군침 자 이제 응용 요리 처음 드릴 응용 요리 는바 킹크랩 다리 튀김 노릇 노릇 기름 킹크랩 다리 튀김 칠리소스 쫘악 준 입속 한번 쫘악 거 환상 궁합 대표 모습 예술 다음 드릴 응용 요리 킹크랩 다리 치즈 요 치즈 다리 주욱 입 말로 치즈 구이 만 준비 킹크랩 방송 분 기대 상품 준비 이번 크리스마스 특집 소개 킹크랩 방송 이번 크리스마스 파티 킹크랩 방송 주제 방송 방송 이번 방송 통해 다음 방송 방송 구성', '품목 레드 킹크랩 섹션 냉동 방식 선상 급냉 제품 상태 자숙 냉동 율 포장 공급 수량 컨테이너 톤 컨 니 선적 사이즈 검품 선적 블라디 보스 톡 검품 선적 가격 지불 조건 계약 체결 시 선금 발급 잔액 지불 수출 회사', '킹크랩 섹션 선상 자숙후 선동 제품 킹크랩 러시아 해역 포획 레드 킹크랩 배 손질 찜 배 냉동 제품 선도 율 최상급 섹션 제품 부위 모두 제거 찜 수령 찜 상태 무게 이기 물상 태의 무게 섹션 키 면 물 이상 자숙 킹크랩 섹션

## 4. 각각의 포스트에 TF-IDF 적용

In [144]:
for i in reversed(range(len(removed_post_2nd))):
    if(len(removed_post_2nd[i]) == 0):
        del removed_post_2nd[i]

In [145]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

corpus = removed_post_2nd

# ============================================
# -- Get TFIDF
# ============================================
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(corpus)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

data = [];
for i, sent in enumerate(corpus):
    print('====== document[%d] ======' % i)
    data.append([ (token, sp_matrix[i, word2id[token]]) for token in sent.split()])
    print(data[i])

====== document[0] ======
[('거성', 0.21457837269201271), ('수산', 0.15582853123292284), ('판매', 0.2171981725989347), ('킹크랩', 0.3436814968373398), ('섹션', 0.12888056131400244), ('러시아', 0.08099055469220981), ('캄차카', 0.16877210698316356), ('해역', 0.12226348977309065), ('포획', 0.11407023579784949), ('하자', 0.13261613680177547), ('마자', 0.13261613680177547), ('배', 0.0), ('킹크랩', 0.3436814968373398), ('손질', 0.08099055469220981), ('자숙', 0.07791426561646142), ('배', 0.0), ('급속', 0.12226348977309065), ('선동', 0.13261613680177547), ('제품', 0.2895975634652463), ('율과', 0.13261613680177547), ('선도', 0.1015043511872706), ('발생', 0.13261613680177547), ('여유', 0.14669019450955137), ('제품', 0.2895975634652463), ('국내', 0.13261613680177547), ('반입', 0.14669019450955137), ('제품', 0.2895975634652463), ('킹크랩', 0.3436814968373398), ('부위', 0.28938049986116227), ('부위', 0.28938049986116227), ('모두', 0.07791426561646142), ('제거', 0.1015043511872706), ('무게', 0.13261613680177547), ('이기', 0.11407023579784949), ('킹크랩', 0.343681496837339

[('거성', 0.21457837269201271), ('수산', 0.15582853123292284), ('판매', 0.2171981725989347), ('킹크랩', 0.3436814968373398), ('섹션', 0.12888056131400244), ('러시아', 0.08099055469220981), ('캄차카', 0.16877210698316356), ('해역', 0.12226348977309065), ('포획', 0.11407023579784949), ('하자', 0.13261613680177547), ('마자', 0.13261613680177547), ('배', 0.0), ('킹크랩', 0.3436814968373398), ('손질', 0.08099055469220981), ('자숙', 0.07791426561646142), ('배', 0.0), ('급속', 0.12226348977309065), ('선동', 0.13261613680177547), ('제품', 0.2895975634652463), ('율과', 0.13261613680177547), ('선도', 0.1015043511872706), ('발생', 0.13261613680177547), ('여유', 0.14669019450955137), ('제품', 0.2895975634652463), ('국내', 0.13261613680177547), ('반입', 0.14669019450955137), ('제품', 0.2895975634652463), ('킹크랩', 0.3436814968373398), ('부위', 0.28938049986116227), ('부위', 0.28938049986116227), ('모두', 0.07791426561646142), ('제거', 0.1015043511872706), ('무게', 0.13261613680177547), ('이기', 0.11407023579784949), ('킹크랩', 0.3436814968373398), ('선어', 0.1072891863460

[('노량진', 0.10365734679390484), ('사랑', 0.07509243788078432), ('수산', 0.047853796433338626), ('오늘', 0.04610109261672507), ('킹크랩', 0.21108412249584788), ('및', 0.0), ('냉동', 0.09220218523345014), ('제품', 0.177866565769553), ('소개', 0.07509243788078432), ('수조', 0.09009496080450545), ('킹크랩', 0.21108412249584788), ('놈', 0.0), ('마리', 0.2161246178094267), ('싱싱', 0.10365734679390484), ('킹크랩', 0.21108412249584788), ('워낙', 0.10365734679390484), ('살기', 0.10365734679390484), ('물', 0.0), ('온도', 0.20731469358780968), ('도', 0.0), ('유지', 0.07509243788078432), ('그냥', 0.07509243788078432), ('손', 0.0), ('겨울', 0.1801899216090109), ('손시', 0.10365734679390484), ('렵습니', 0.10365734679390484), ('왕따', 0.10365734679390484), ('킹크랩', 0.21108412249584788), ('동료', 0.10365734679390484), ('못', 0.0), ('놈', 0.0), ('먼저', 0.07006026174775283), ('잡기', 0.10365734679390484), ('놈', 0.0), ('마리', 0.2161246178094267), ('포장', 0.2101807852432585), ('아이스팩', 0.09009496080450545), ('놈', 0.0), ('마리', 0.2161246178094267), ('박스', 0.0540311544

109


In [146]:
for i in range(0, len(data)):
    toCSV=pd.DataFrame(data[i])
    toCSV=toCSV.sort_values(by=1,ascending=False)
    toCSV=toCSV.drop_duplicates()
    toCSV.to_csv("data"+str(i)+".csv",encoding='ms949')

[]